In [1]:
//TCP server to OF

const net = require('net');
const port = 7070;
const host = '127.0.0.1';

const server = net.createServer();
server.listen(port, host, () => {
    console.log('TCP Server is running on port ' + port + '.');
});

let sockets = [];

server.on('connection', function(sock) {
    console.log('CONNECTED: ' + sock.remoteAddress + ':' + sock.remotePort);
    sockets.push(sock);

    sock.on('data', function(data) {
        console.log('DATA ' + sock.remoteAddress + ': ' + data);
        // Write the data back to all the connected, the client will receive it as data from the server
        sockets.forEach(function(sock, index, array) {
            sock.write(sock.remoteAddress + ':' + sock.remotePort + " said " + data + '\n');
        });
    });

    // Add a 'close' event handler to this instance of socket
    sock.on('close', function(data) {
        let index = sockets.findIndex(function(o) {
            return o.remoteAddress === sock.remoteAddress && o.remotePort === sock.remotePort;
        })
        if (index !== -1) sockets.splice(index, 1);
        console.log('CLOSED: ' + sock.remoteAddress + ' ' + sock.remotePort);
    });
});

Server {
  _events: [Object: null prototype] {
    listening: [Function: bound onceWrapper] { listener: [Function (anonymous)] },
    connection: [Function (anonymous)]
  },
  _eventsCount: 2,
  _maxListeners: undefined,
  _connections: 0,
  _handle: null,
  _usingWorkers: false,
  _workers: [],
  _unref: false,
  allowHalfOpen: false,
  pauseOnConnect: false,
  [Symbol(kCapture)]: false,
  [Symbol(async_id_symbol)]: -1
}

In [2]:
//OSC Server to touchOSC

const osc = require('osc');

const OSC_PORT = 7071

let tcp_osc_port = new osc.TCPSocketPort({});
// change to remote host/port (address of phone/tablet) - set touchOSC on device to TCPserver
tcp_osc_port.open('192.168.1.39', OSC_PORT); 

tcp_osc_port.on('ready', () => {
  console.log('ready TCP');
});

// Bind to a UDP socket to listen for incoming OSC events.
let udp_osc_port = new osc.UDPPort({
    localAddress: "0.0.0.0",
    localPort: OSC_PORT
});

udp_osc_port.on("ready", () => {
    console.log('ready OSC');
});

udp_osc_port.open();

let osc_port = udp_osc_port;

const rawLoops = {};
const processedLoops = {};
let activeLoop = 'firstLoop';
let isRecording = false;
let isTouching = false;
const handlers = {};

function processLoop(loopKey) {
    processedLoops[loopKey] = rawLoops[loopKey];
}

handlers['/recording'] = (isRec) => {
    isRecording = isRec;
    if(isRecording) {
        rawLoops[activeLoop] = [{ts: Date.now(), pos: 'start'}];
    } else {
        rawLoops[activeLoop].push({ts: Date.now(), pos: 'end'});
        processLoop(activeLoop);
        sockets.forEach(sock => sock.write( JSON.stringify(processedLoops) + '\n' ))
    }
}
handlers['/pos'] = (x, y) => {
    if(isRecording) {
        rawLoops[activeLoop].push({ts: Date.now(), pos: {x, y}});
    }
    //todo: send pos to OF for visualization of live touch point
}
handlers['/pos/touch'] = (touching) => {
    isTouching = touching;
    //todo: send isTouching to OF for visualization of live touch point
}

osc_port.on('message', (message) => {
    let {address, args} = message;

    if(handlers[address]) handlers[address](...args); 
});



TCP Server is running on port 7070.


EventEmitter {
  options: { useSLIP: true, address: '127.0.0.1', port: 57121 },
  decoder: {
    maxMessageSize: 10485760,
    bufferSize: 1024,
    msgBuffer: Uint8Array(1024) [
      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
      0, 0, 0, 0,
      ... 924 more items
    ],
    msgBufferIdx: 0,
    onMessage: [Function: bound ],
    onError: [Function: onError],
    escape: false
  },
  _events: [Object: null prototype] {
    data: [Function: bound ],
    open: [Function: bound ],
    ready: [Function (anonymous)],
    message: [Function (anonymous)]
  },
  _eventsCount: 4,
  socket: <ref *1> Socket {
    connecting: true,
    _hadError: false,
    _parent: null,
    _host: null,
    _readableState: R

In [4]:
rawLoops

{}